# Ejercicio 3 — Búsqueda BLAST a partir de un fichero FASTA

En este ejercicio se desarrolla un programa en Python que realiza una búsqueda BLAST a partir
de una secuencia almacenada en un fichero en formato FASTA.

El programa realiza la búsqueda de dos maneras:

- **De forma online**, usando los servidores del NCBI.
- **De forma local**, utilizando BLAST+ instalado en el sistema.

En ambos casos, los resultados son filtrados por un organismo concreto, y finalmente se muestran:

1. El número de resultados obtenidos tras el filtrado.
2. El E-value medio de dichos resultados.

Esto permite evaluar el grado de similitud de la secuencia analizada con secuencias
conocidas de un organismo específico.


In [1]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import Entrez, SeqIO
import tempfile
import os
import numpy as np


## 1. Lectura de la secuencia desde un fichero FASTA

En lugar de introducir la secuencia manualmente, el programa la lee desde un fichero FASTA.  
De esta manera se trabaja como en un entorno real de bioinformática, donde las secuencias
se almacenan en ficheros estructurados.

In [2]:
# CONFIGURACIÓN USUARIO

Entrez.email = "mariana.bordes101@alu.ulpgc.es"

ARCHIVO_FASTA = "secuencia.fasta"
ORGANISMO = "Homo sapiens"   # Cambiar si se desea
DB = "nt"


In [3]:
# CREAR FASTA (SI NO EXISTE)

if not os.path.exists(ARCHIVO_FASTA):
    with open(ARCHIVO_FASTA, "w") as f:
        f.write(">mi_secuencia\n")
        f.write("ATGCGTACGATCGATCGATCGATCGTTAGC\n")
else:
    print('Fasta file found!')

In [4]:
# CARGA DE SECUENCIA FASTA

record = SeqIO.read(ARCHIVO_FASTA, "fasta")
secuencia = str(record.seq)

print("Secuencia leída desde FASTA:")
print(secuencia)

Secuencia leída desde FASTA:
ATGCGTACGATCGATCGATCGATCGTTAGC


In [5]:
# FUNCIÓN DE FILTRADO POR ORGANISMO Y EVALUE MEDIO

def filtrar_resultados(record, organismo):
    evalues = []
    
    for alignment in record.alignments:
        if organismo.lower() in alignment.title.lower():
            for hsp in alignment.hsps:
                evalues.append(hsp.expect)
    
    return len(evalues), np.mean(evalues) if evalues else 0

## 2. Búsqueda BLAST online

Se realiza una búsqueda BLASTn enviando la secuencia al servidor del NCBI.  
Los resultados se parsean en formato XML y se filtran por organismo.

Finalmente se calcula:

- Número de resultados filtrados.
- E-value medio.


In [6]:
# EJECUCIÓN BLAST ONLINE

print("Ejecutando BLAST online...")

handle = NCBIWWW.qblast(
    program="blastn",
    database=DB,
    sequence=secuencia
)

blast_record_online = NCBIXML.read(handle)

Ejecutando BLAST online...


In [7]:
# RESULTADOS ONLINE

num_online, evalue_online = filtrar_resultados(blast_record_online, ORGANISMO)

print("\n--- RESULTADOS BLAST ONLINE ---")
print("Organismo:", ORGANISMO)
print("Número de resultados:", num_online)
print(f"E-value medio: {evalue_online:.2e}")
print("-------------------------------")


--- RESULTADOS BLAST ONLINE ---
Organismo: Homo sapiens
Número de resultados: 0
E-value medio: 0.00e+00
-------------------------------


## 3. Búsqueda BLAST local

En esta parte se ejecuta BLAST directamente en el sistema.

Es necesario:
- Tener BLAST+ instalado.
- Tener una base de datos creada previamente usando `makeblastdb`.

La ejecución genera un XML que posteriormente se analiza igual que en BLAST online.


In [ ]:
# EJECUCIÓN BLAST LOCAL 

BLASTN_EXE = "blastn"
DB_LOCAL = "mi_db_nucl"

with tempfile.TemporaryDirectory() as tmpdir:
    fasta_path = os.path.join(tmpdir, "query.fasta")
    xml_out = os.path.join(tmpdir, "blast_local.xml")
    
    with open(fasta_path, "w") as f:
        f.write(">consulta\n")
        f.write(secuencia + "\n")

    blast_cline = NcbiblastnCommandline(
        cmd=BLASTN_EXE,
        query=fasta_path,
        db=DB_LOCAL,
        outfmt=5,
        out=xml_out
    )

    stdout, stderr = blast_cline()

    with open(xml_out) as result_handle:
        blast_record_local = NCBIXML.read(result_handle)

ApplicationError: Non-zero return code 3221225477 from 'blastn -out C:\\Users\\mariana\\AppData\\Local\\Temp\\tmpgjzkhg4q\\blast_local.xml -outfmt 5 -query C:\\Users\\mariana\\AppData\\Local\\Temp\\tmpgjzkhg4q\\query.fasta -db mi_db_nucl'

In [ ]:
# RESULTADOS BLAST LOCAL

num_local, evalue_local = filtrar_resultados(blast_record_local, ORGANISMO)

print("\n--- RESULTADOS BLAST LOCAL ---")
print("Organismo:", ORGANISMO)
print("Número de resultados:", num_local)
print(f"E-value medio: {evalue_local:.2e}")
print("-------------------------------")

## Conclusiones

- El BLAST online permite acceder a bases de datos actualizadas sin instalación.
- El BLAST local es más rápido y permite trabajar sin conexión.
- El uso de filtros por organismo permite centrar el análisis en especies de interés.
- El E-value medio permite evaluar la calidad global de los alineamientos.

Este ejercicio demuestra cómo automatizar consultas BLAST usando Python y Biopython
tanto en modo remoto como local.
